In [2]:
# import os
# os.chdir('../')
# %pwd

'/Users/abrar/ML-DS-Projects/Practice/End-to-End-ML-Project-with-MLOps'

In [3]:
import dagshub

dagshub.init(repo_owner='abrarshahok', repo_name='End-to-End-ML-Project-with-MLOps', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=02b36b5a-9da2-4e2d-9773-b3fcbc0eadbe&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1d0de8db5252b0dc071c8625825c52ddf8935822b44ed50ce3bae68a036bc41b




Accessing as abrarshahok

Initialized MLflow to track repo "abrarshahok/End-to-End-ML-Project-with-MLOps"

Repository abrarshahok/End-to-End-ML-Project-with-MLOps initialized!

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metrics_path: Path
    target_column: str
    experiment_name: str

In [50]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):   
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metrics_path=config.evaluation_metrics_path,
            target_column=schema.name,
            experiment_name=config.experiment_name
        )

        return model_evaluation_config


In [46]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
from src.mlProject.utils.common import save_json
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [53]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, preds):
        rmse = np.sqrt(mean_squared_error(actual, preds))
        mae = mean_absolute_error(actual, preds)
        r2 = r2_score(actual, preds)
        return rmse, mae, r2

    def log_into_mlflow(self):
        experiment = mlflow.get_experiment_by_name(self.config.experiment_name)
        if experiment is None:
            mlflow.create_experiment(self.config.experiment_name)
        mlflow.set_experiment(self.config.experiment_name)
    
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(self.config.target_column, axis=1)
        test_y = test_data[self.config.target_column]

        tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():
            preds = model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_metrics(test_y, preds)

            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }

            save_json(path=Path(self.config.metrics_path), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")



In [54]:
try:
    config_mgr = ConfigurationManager()
    model_evaluation_config = config_mgr.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-04 04:44:52,240: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-04 04:44:52,247: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-04 04:44:52,254: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-04 04:44:52,262: INFO: common: created directory at: artifacts]
[2025-01-04 04:44:52,263: INFO: common: created directory at: artifacts/model_evaluation]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/abrar/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/j2/x1x7xxzs76lfwxs0nvtq0hbm0000gn/T/ipykernel_20593/2621622602.py", line 7, in <module>
    raise e
  File "/var/folders/j2/x1x7xxzs76lfwxs0nvtq0hbm0000gn/T/ipykernel_20593/2621622602.py", line 5, in <module>
    model_evaluation.log_into_mlflow()
  File "/var/folders/j2/x1x7xxzs76lfwxs0nvtq0hbm0000gn/T/ipykernel_20593/3840595933.py", line 12, in log_into_mlflow
    experiment = mlflow.get_experiment_by_name(self.config.experiment_name)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'ModelEvaluationConfig' object has no attribute 'experiment_name'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/abrar/Library/Python/3.12/lib/python/site-packag